In [6]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
# from dataset.data_loader import GetLoader
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from skimage import io, transform
import cv2
import numpy as np
import pandas as pd


class MNIST(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """ Intialize the MNIST dataset """
        self.root_dir = root_dir
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[index,0])
        image = io.imread(img_name)
        # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        label = self.landmarks_frame['label'][index]
        label = torch.FloatTensor([label])

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return len(self.landmarks_frame)

def test(dataset_name, epoch):
    assert dataset_name in ['MNIST', 'SVHN']

    model_root = os.path.join('.','model')
    image_root = os.path.join('..', 'dataset', dataset_name)

    cuda = True
    cudnn.benchmark = True
    batch_size = 128
    image_size = 28
    alpha = 0

    """load data"""

    # img_transform_source = transforms.Compose([
    #     transforms.Resize(image_size),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
    # ])

    # img_transform_target = transforms.Compose([
    #     transforms.Resize(image_size),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    # ])

    if dataset_name == 'SVHN':
        # test_list = os.path.join(image_root, 'mnist_m_test_labels.txt')

        # dataset = GetLoader(
        #     data_root=os.path.join(image_root, 'mnist_m_test'),
        #     data_list=test_list,
        #     transform=img_transform_target
        # )
        dataset =  MNIST(csv_file="../../hw3_data/digits/svhn/test.csv", root_dir="../../hw3_data/digits/svhn/test",transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
    else:
        dataset = MNIST(csv_file="../../hw3_data/digits/mnistm/test.csv", root_dir="../../hw3_data/digits/mnistm/test",transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

    dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8
    )

    """ training """

    my_net = torch.load(os.path.join(
        model_root, 'mnist_svhn_model' + str(epoch) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0
#     features = []
    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label.long()).copy_(t_label.long())

        class_output, _,_ = my_net(input_data=input_img, alpha=alpha)
#         features.append(feature.cpu())
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))
#     return features

In [10]:
for i in range(17):
    test("SVHN",i)

epoch: 0, accuracy of the SVHN dataset: 0.451444
epoch: 1, accuracy of the SVHN dataset: 0.453634
epoch: 2, accuracy of the SVHN dataset: 0.436540
epoch: 3, accuracy of the SVHN dataset: 0.436616
epoch: 4, accuracy of the SVHN dataset: 0.456592
epoch: 5, accuracy of the SVHN dataset: 0.448832
epoch: 6, accuracy of the SVHN dataset: 0.454364
epoch: 7, accuracy of the SVHN dataset: 0.473725
epoch: 8, accuracy of the SVHN dataset: 0.481446
epoch: 9, accuracy of the SVHN dataset: 0.474493
epoch: 10, accuracy of the SVHN dataset: 0.482253
epoch: 11, accuracy of the SVHN dataset: 0.481484
epoch: 12, accuracy of the SVHN dataset: 0.476490
epoch: 13, accuracy of the SVHN dataset: 0.445337
epoch: 14, accuracy of the SVHN dataset: 0.471766
epoch: 15, accuracy of the SVHN dataset: 0.494968
epoch: 16, accuracy of the SVHN dataset: 0.494199


In [12]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
# from dataset.data_loader import GetLoader
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from skimage import io, transform
import cv2
import numpy as np
import pandas as pd

class MNIST(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """ Intialize the MNIST dataset """
        self.root_dir = root_dir
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[index,0])
        image = io.imread(img_name)
        # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        label = self.landmarks_frame['label'][index]
        label = torch.FloatTensor([label])

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return len(self.landmarks_frame)

class USPS(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """ Intialize the MNIST dataset """
        self.root_dir = root_dir
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[index,0])
        image = io.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        label = self.landmarks_frame['label'][index]
        label = torch.FloatTensor([label])

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return len(self.landmarks_frame)

def test(dataset_name, epoch):
    assert dataset_name in ['USPS', 'MNIST']

    model_root = os.path.join('.','model')
    image_root = os.path.join('..', 'dataset', dataset_name)

    cuda = True
    cudnn.benchmark = True
    batch_size = 128
    image_size = 28
    alpha = 0

    """load data"""

    # img_transform_source = transforms.Compose([
    #     transforms.Resize(image_size),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
    # ])

    # img_transform_target = transforms.Compose([
    #     transforms.Resize(image_size),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    # ])

    if dataset_name == 'MNIST':
        # test_list = os.path.join(image_root, 'mnist_m_test_labels.txt')

        # dataset = GetLoader(
        #     data_root=os.path.join(image_root, 'mnist_m_test'),
        #     data_list=test_list,
        #     transform=img_transform_target
        # )
        dataset =  MNIST(csv_file="../../hw3_data/digits/mnistm/test.csv", root_dir="../../hw3_data/digits/mnistm/test",transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
    else:
        dataset = USPS(csv_file="../../hw3_data/digits/usps/test.csv", root_dir="../../hw3_data/digits/usps/test",transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

    dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8
    )

    """ training """

    my_net = torch.load(os.path.join(
        model_root, 'usps_mnist_model' + str(epoch) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label.long()).copy_(t_label.long())

        class_output, _,_ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))


In [13]:
for i in range(15):
    test("MNIST",i)

/home/robot/anaconda3/envs/env/lib/python3.7/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'model.CNNModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


epoch: 0, accuracy of the MNIST dataset: 0.304300
epoch: 1, accuracy of the MNIST dataset: 0.361600
epoch: 2, accuracy of the MNIST dataset: 0.348500
epoch: 3, accuracy of the MNIST dataset: 0.367800
epoch: 4, accuracy of the MNIST dataset: 0.384300
epoch: 5, accuracy of the MNIST dataset: 0.388300
epoch: 6, accuracy of the MNIST dataset: 0.377500
epoch: 7, accuracy of the MNIST dataset: 0.408700
epoch: 8, accuracy of the MNIST dataset: 0.387300
epoch: 9, accuracy of the MNIST dataset: 0.412800
epoch: 10, accuracy of the MNIST dataset: 0.414500
epoch: 11, accuracy of the MNIST dataset: 0.396600
epoch: 12, accuracy of the MNIST dataset: 0.377900
epoch: 13, accuracy of the MNIST dataset: 0.407300
epoch: 14, accuracy of the MNIST dataset: 0.406500


In [15]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
# from dataset.data_loader import GetLoader
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from skimage import io, transform
import cv2
import numpy as np
import pandas as pd

class MNIST(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """ Intialize the MNIST dataset """
        self.root_dir = root_dir
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[index,0])
        image = io.imread(img_name)
        # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        label = self.landmarks_frame['label'][index]
        label = torch.FloatTensor([label])

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return len(self.landmarks_frame)

class USPS(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """ Intialize the MNIST dataset """
        self.root_dir = root_dir
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[index,0])
        image = io.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        label = self.landmarks_frame['label'][index]
        label = torch.FloatTensor([label])

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return len(self.landmarks_frame)

def test(dataset_name, epoch):
    assert dataset_name in ['SVHN', 'USPS']

    model_root = os.path.join('.','model')
    image_root = os.path.join('..', 'dataset', dataset_name)

    cuda = True
    cudnn.benchmark = True
    batch_size = 128
    image_size = 28
    alpha = 0

    """load data"""

    # img_transform_source = transforms.Compose([
    #     transforms.Resize(image_size),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
    # ])

    # img_transform_target = transforms.Compose([
    #     transforms.Resize(image_size),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    # ])

    if dataset_name == 'USPS':
        # test_list = os.path.join(image_root, 'mnist_m_test_labels.txt')

        # dataset = GetLoader(
        #     data_root=os.path.join(image_root, 'mnist_m_test'),
        #     data_list=test_list,
        #     transform=img_transform_target
        # )
        dataset =  USPS(csv_file="../../hw3_data/digits/usps/test.csv", root_dir="../../hw3_data/digits/usps/test",transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
    else:
        dataset = MNIST(csv_file="../../hw3_data/digits/svhn/test.csv", root_dir="../../hw3_data/digits/svhn/test",transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

    dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8
    )

    """ training """

    my_net = torch.load(os.path.join(
        model_root, 'svhn_usps_model' + str(epoch) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label.long()).copy_(t_label.long())

        class_output, _,_ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))


In [16]:
for i in range(15):
    test("USPS",i)

epoch: 0, accuracy of the USPS dataset: 0.257598
epoch: 1, accuracy of the USPS dataset: 0.343298
epoch: 2, accuracy of the USPS dataset: 0.470852
epoch: 3, accuracy of the USPS dataset: 0.560538
epoch: 4, accuracy of the USPS dataset: 0.565521
epoch: 5, accuracy of the USPS dataset: 0.605879
epoch: 6, accuracy of the USPS dataset: 0.574988
epoch: 7, accuracy of the USPS dataset: 0.514200
epoch: 8, accuracy of the USPS dataset: 0.558545
epoch: 9, accuracy of the USPS dataset: 0.520678
epoch: 10, accuracy of the USPS dataset: 0.516692
epoch: 11, accuracy of the USPS dataset: 0.431988
epoch: 12, accuracy of the USPS dataset: 0.404086
epoch: 13, accuracy of the USPS dataset: 0.338316
epoch: 14, accuracy of the USPS dataset: 0.497758


In [13]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
# from dataset.data_loader import GetLoader
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from skimage import io, transform
import cv2
import numpy as np
import pandas as pd


class MNIST(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """ Intialize the MNIST dataset """
        self.root_dir = root_dir
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform
                              
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[index,0])
        image = io.imread(img_name)
        # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        label = self.landmarks_frame['label'][index]
        label = torch.FloatTensor([label])

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return len(self.landmarks_frame)

# def test(dataset_name, epoch):
dataset_name = 'SVHN'
epoch = 10
assert dataset_name in ['MNIST', 'SVHN']

model_root = os.path.join('.','model')
image_root = os.path.join('..', 'dataset', dataset_name)

cuda = True
cudnn.benchmark = True
batch_size = 128
image_size = 28
alpha = 0

"""load data"""

# img_transform_source = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
# ])

# img_transform_target = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
# ])

if dataset_name == 'SVHN':
    # test_list = os.path.join(image_root, 'mnist_m_test_labels.txt')

    # dataset = GetLoader(
    #     data_root=os.path.join(image_root, 'mnist_m_test'),
    #     data_list=test_list,
    #     transform=img_transform_target
    # )
    dataset =  MNIST(csv_file="../../hw3_data/digits/svhn/test.csv", root_dir="../../hw3_data/digits/svhn/test",transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ]))
else:
    dataset = MNIST(csv_file="../../hw3_data/digits/mnistm/test.csv", root_dir="../../hw3_data/digits/mnistm/test",transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ]))

dataloader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8
)

""" training """

my_net = torch.load(os.path.join(
    model_root, 'mnist_svhn_model' + str(epoch) + '.pth'
))
my_net = my_net.eval()

if cuda:
    my_net = my_net.cuda()

len_dataloader = len(dataloader)
data_target_iter = iter(dataloader)

i = 0
n_total = 0
n_correct = 0
predict = []
while i < len_dataloader:

    # test model using target data
    data_target = data_target_iter.next()
    t_img, t_label = data_target

    batch_size = len(t_label)

    input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
    class_label = torch.LongTensor(batch_size)

    if cuda:
        t_img = t_img.cuda()
        t_label = t_label.cuda()
        input_img = input_img.cuda()
        class_label = class_label.cuda()

    input_img.resize_as_(t_img).copy_(t_img)
    class_label.resize_as_(t_label.long()).copy_(t_label.long())

    class_output, _,_ = my_net(input_data=input_img, alpha=alpha)
#         features.append(feature.cpu())
    pred = class_output.data.max(1, keepdim=True)[1]

    n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
    predict += pred.squeeze().cpu().tolist()
    n_total += batch_size

    i += 1

accu = n_correct.data.numpy() * 1.0 / n_total

print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))


epoch: 10, accuracy of the SVHN dataset: 0.482253


In [14]:
predict

[9,
 9,
 1,
 0,
 9,
 1,
 3,
 1,
 1,
 8,
 3,
 5,
 9,
 1,
 9,
 4,
 1,
 0,
 3,
 4,
 2,
 0,
 1,
 9,
 1,
 9,
 0,
 1,
 4,
 7,
 8,
 5,
 3,
 5,
 4,
 1,
 5,
 1,
 7,
 2,
 3,
 3,
 0,
 3,
 9,
 0,
 1,
 2,
 9,
 6,
 9,
 1,
 5,
 4,
 8,
 6,
 4,
 5,
 1,
 8,
 2,
 4,
 9,
 1,
 3,
 9,
 6,
 9,
 4,
 2,
 3,
 5,
 7,
 1,
 0,
 8,
 8,
 9,
 3,
 1,
 1,
 4,
 7,
 8,
 2,
 1,
 1,
 1,
 0,
 1,
 4,
 6,
 8,
 4,
 4,
 1,
 1,
 1,
 2,
 7,
 5,
 8,
 2,
 6,
 7,
 2,
 4,
 1,
 3,
 5,
 3,
 8,
 5,
 4,
 0,
 7,
 2,
 2,
 3,
 8,
 8,
 8,
 1,
 1,
 4,
 9,
 0,
 4,
 1,
 8,
 6,
 5,
 5,
 3,
 3,
 8,
 9,
 1,
 3,
 0,
 1,
 2,
 9,
 6,
 1,
 2,
 1,
 2,
 3,
 3,
 1,
 5,
 9,
 1,
 5,
 9,
 3,
 0,
 6,
 9,
 0,
 4,
 3,
 0,
 9,
 3,
 9,
 9,
 0,
 8,
 1,
 8,
 3,
 2,
 2,
 0,
 0,
 8,
 5,
 7,
 1,
 9,
 7,
 2,
 4,
 8,
 8,
 8,
 7,
 8,
 8,
 6,
 1,
 9,
 3,
 3,
 0,
 1,
 1,
 3,
 2,
 7,
 7,
 0,
 0,
 2,
 1,
 6,
 3,
 7,
 1,
 3,
 1,
 1,
 7,
 3,
 1,
 1,
 1,
 2,
 7,
 6,
 7,
 1,
 1,
 0,
 7,
 2,
 2,
 9,
 2,
 5,
 7,
 0,
 1,
 1,
 0,
 6,
 3,
 2,
 2,
 2,
 8,
 4,
 1,
 1,
 4,
 0,
 3,
 5,


In [15]:
pred

tensor([[5],
        [7],
        [5],
        [2],
        [4],
        [5],
        [0],
        [0],
        [1],
        [1],
        [5],
        [1],
        [8],
        [2],
        [9],
        [1],
        [3],
        [1],
        [6],
        [4],
        [7],
        [8],
        [8],
        [1],
        [5],
        [9],
        [7],
        [0],
        [1],
        [8],
        [8],
        [3],
        [4],
        [2],
        [9],
        [8],
        [1],
        [5],
        [1],
        [7],
        [1],
        [4],
        [8],
        [8],
        [2],
        [7],
        [0],
        [1]], device='cuda:0')